## Ceneo Scraper

## załadowanie bibliotek


In [3]:
import requests
from bs4 import BeautifulSoup

## wysyłanie zapytania do serwera

In [4]:
product_id = "138331381"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
response = requests.get(url)
response.status_code



200

In [10]:
page_dom = BeautifulSoup(response.text, "html.parser")
print(type(page_dom))
opinions = page_dom.select("div.js_product-review")
print(type(opinions))
opinion = opinions.pop(0)


<class 'bs4.BeautifulSoup'>
<class 'bs4.element.ResultSet'>


## Ekstrakcje składowych pojedynczej opinii
|Składowa|Selektor|zmienna|
|--------|--------|-------|
|id opinii|[data-entry-id]| opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation|recommendation|
|gwiazdki|span.user-post__score-count|stars|
|treść|div.user-post__text|contents|
|lista zalet|div.review-feature__title--positives ~ div.review-feature_item|pros|
|lisita wad|div.review-feature__title--negatives ~ div.review-feature_item|cons| 
|dla ilu przydatna|button.vote-yes > span|helpful|
|dla ilu nieprzydatna| button.vote-no > span|unhelpful|
|data wystawienia|span.user-post_published > time:nth-child(1)["datetime"]|publish_date|
|data zakupu|span.user-post_published > time:nth-child(2)["datetime"]|purchase_date|


In [ ]:
opinion_id = opinion["div.data-entry-id"]
author = opinion.select_one("div.user-post__author-name")
recommendation = opinion.select_one("div.user-post__author-recomendation")
stars = opinion.select_one("div.user-post__score-count")
contents = opinion.select_one("div.user-post__text")
pros = opinion.select_one("div.review-feature__title review-feature__title--positives")
cons = opinion.select_one("div.review-feature__title review-feature__title--")
helpful = opinion.select_one("div.js_product-review-usefulness vote")
unhelpful = opinion.select_one("div.user-post__author-name")
publish_date = opinion.select_one("div.user-post__author-name")
purchase_date =  opinion.select_one("div.user-post__author-name")